In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [9]:
x_train = torch.tensor([[1,2],
                       [3,4],
                       [5,6],
                       [7,8]])
y_train = torch.tensor([0, 1, 0, 1])
x_test = torch.tensor([[9,10],
                      [11,12],
                      [13,14],
                      [15,16]])
y_test = torch.tensor([0, 1, 0, 1])

传入数据的类型可以是  
- `torch.Tensor`（如：`x_train`, `y_train`, `x_test`, `y_test`）  
- 自定义的 `Dataset` 类（如：`train_dataset`, `test_dataset`，对应变量 `tr`, `te`）

对于自定义的 `Dataset` 类，传入的数据不一定非要是 `torch.Tensor`，也可以是 `numpy.ndarray`、`list` 等类型。只要在 `__getitem__` 方法中能够正确处理这些数据类型即可。不过，为了与 PyTorch 的训练流程兼容，通常推荐使用 `torch.Tensor`。

In [ ]:
from torch.utils.data import Dataset
class train_dataset(Dataset):
    def __init__(self, x, y, transform=True):
        self.feature = x
        self.label = y
        if transform:
            self.transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(10),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
            ])

    def __len__(self):
        return len(self.feature)

    def __getitem__(self, idx):
        return self.feature[idx], self.label[idx]
    #   return self.transform(self.feature[idx]), self.label[idx]

class test_dataset(Dataset):
    def __init__(self, x, y):
        self.feature = x
        self.label = y
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
    def __len__(self):
        return len(self.feature)
    def __getitem__(self, idx):
        return {'feature': self.feature[idx],
                'label': self.label[idx]}


构建自定义数据类

In [11]:
tr = train_dataset(x_train, y_train, transform=True)
te = test_dataset(x_test, y_test)

In [12]:
tr.__len__()  # Should return 4

4

In [ ]:
from torch.utils.data import DataLoader
torch.manual_seed(0) # For reproducibility
train_loader = DataLoader(tr, batch_size=2, shuffle=True,num_workers=0, drop_last=True)
test_loader = DataLoader(te, batch_size=2, shuffle=False,num_workers=0)
for idx,(x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:")
    print("Features:", x)
    print("Labels:", y)


Batch 1:
Features: tensor([[5, 6],
        [3, 4]])
Labels: tensor([0, 1])
Batch 2:
Features: tensor([[7, 8],
        [1, 2]])
Labels: tensor([1, 0])


将实例化的数据集传入dataloader  
如果样本数不能被batch整除，加上drop_last，否则最后多出来的样本会有更大的权重不平衡  
`num_workers` 参数用于设置加载数据时使用的子进程数量，可以加速数据的读取。它不是用来分配内存的，而是通过多进程并行预取数据，提高数据加载效率。比如worker为0时，只有一个进程他需要训练一组数据，然后在加载另一组数据，中间就会花费一些时间，但是如果有两个worker，就可以一个在训练另一个同步加载数据，节省时间。

worker数量增加的代价主要有：  
- 占用更多的CPU资源，可能导致系统资源紧张，影响其他进程  
- 每个worker进程都需要初始化，会有一定的启动开销  
- 数据加载过程中，主进程和worker进程之间需要进行数据的通信（如通过队列传递），这会带来一定的内存和带宽消耗  
- 如果数据预处理很轻量，worker过多反而可能导致效率下降（进程切换、通信开销大于实际加速效果）

In [ ]:
# on the fly data loading (more efficient)
import os
from PIL import Image
class largeimage_dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = os.listdir(root_dir) # return list of files in the directory

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB') # get image here, one by one
        if self.transform:
            image = self.transform(image)
        return image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])